In [ ]:
!pip install keras-tuner

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import save_model, load_model
import joblib
import matplotlib.pyplot as plt
from tensorflow import keras
from kerastuner import RandomSearch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
training_df = pd.read_csv('/content/drive/MyDrive/FYP/NN/BTCUSDT/BTCUSDT_elliot_dataset.csv')

In [ ]:
def process_training_data(df):
  # Shuffle the DataFrame
  df = df.sample(frac=1).reset_index(drop=True)

  # 2. Extract features (X) and labels (y)
  # Assuming the last column contains labels and the rest are features
  X = df.iloc[:, :-1].values
  y = df.iloc[:, -1].values
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\
  scaler = MinMaxScaler(feature_range=(0.01, 0.99))
  X_train_scaled = scaler.fit_transform(X_train)
  X_test_scaled = scaler.transform(X_test)

  return X_train_scaled, X_test_scaled, y_train, y_test, scaler

In [ ]:
def model_builder(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_1', min_value=32, max_value=128, step=32), activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(units=hp.Int('units_2', min_value=16, max_value=64, step=16), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=keras.optimizers.Adam(hp.Float('learning_rate', min_value=0.001, max_value=0.01, step=0.001)),
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
def hyperparameter_tuning_get_best_model(model_builder, X_train_scaled, y_train,X_test_scaled, y_test):
  tuner = RandomSearch(model_builder,
                     objective='val_accuracy',
                     max_trials=10,
                     executions_per_trial=2,
                     directory='my_dir',
                     project_name='btcusdt_nn_tuning')

  tuner.search(X_train_scaled, y_train,
             epochs=50,
             validation_data=(X_test_scaled, y_test))

  best_model = tuner.get_best_hyperparameters()[0]
  best_hyperparameters = tuner.get_best_hyperparameters()[0]
  best_units_1 = best_hyperparameters.get('units_1')
  best_units_2 = best_hyperparameters.get('units_2')
  best_learning_rate = best_hyperparameters.get('learning_rate')

  print("Best Hyperparameters:")
  print(f"Units_1: {best_units_1}")
  print(f"Units_2: {best_units_2}")
  print(f"Learning Rate: {best_learning_rate}")

  return best_model

In [ ]:
def elliot_training(best_model, X_train_scaled, y_train,X_test_scaled, y_test):
  best_model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)
  loss, accuracy = best_model.evaluate(X_test_scaled, y_test)
  print(f'Test Accuracy (Best Model): {accuracy}')

In [ ]:
def save_model(best_model, scaler):
  best_model.save('/content/drive/MyDrive/FYP/NN/BTCUSDT/neural_network_for_BTCUSDT_Hype.h5')
  joblib.dump(scaler, '/content/drive/MyDrive/FYP/NN/BTCUSDT/neural_network_scaler_for_BTCUSDT_hype.gz')


In [ ]:
def build_best_model(best_units_1 , best_units_2, best_learning_rate, X_train):
    model = Sequential()
    model.add(Dense(units=best_units_1, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(units=best_units_2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=best_learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

Training with hyperparameter tuning

In [ ]:
def training_with_hyperparameter_tuning():
  X_train_scaled, X_test_scaled, y_train, y_test, scaler = process_training_data(training_df)
  best_model = hyperparameter_tuning_get_best_model(model_builder, X_train_scaled, y_train,X_test_scaled, y_test)
  elliot_training(best_model,X_train_scaled, y_train,X_test_scaled, y_test)
  save_model(best_model, scaler)

Training without hyperparameter tuning


In [ ]:
def training_without_hyperparameter_tuning(training_df):
  X_train_scaled, X_test_scaled, y_train, y_test, scaler = process_training_data(training_df)
  best_model = build_best_model(96, 32, 0.007, X_train_scaled)
  elliot_training(best_model,X_train_scaled, y_train,X_test_scaled, y_test)
  save_model(best_model, scaler)

In [ ]:
# rename file paths to save model, save scaler and to read the training_df
training_without_hyperparameter_tuning(training_df)

NameError: name 'training_without_hyperparameter_tuning' is not defined

Prediction

In [ ]:
# Load the model
loaded_model = load_model('/content/drive/MyDrive/Stock Market Group/Result Analysis/Results/Elliott Wave NN/neural_network_default_new.h5')
# Load the scaler
scaler = joblib.load('/content/drive/MyDrive/Stock Market Group/Result Analysis/Results/Elliott Wave NN/neural_network_scaler_default_new.gz')
# Load data from CSV
df = pd.read_csv('/content/drive/MyDrive/Stock Market Group/Result Analysis/Results/DataSet for 20 /elliot_wave_data_input.csv')

In [ ]:
output_path = '/content/drive/MyDrive/Stock Market Group/Result Analysis/Results/DataSet for 20 /elliot_output_appl_20.csv'

In [ ]:
def plot_most_likely(most_likely):
  # Plotting the list values
  plt.plot(most_likely, marker='o', linestyle='-')

  # Adding labels and title
  plt.xlabel('Index')
  plt.ylabel('Value')
  plt.title('List Plot')

  # Display the plot
  plt.show()

In [ ]:
def elliot_predict(df, scaler, loaded_model, output_path):
  data = df.iloc[:, :9].values
  result_array = np.array(data)
  closed_prices_scaled = scaler.transform(result_array)
  predictions = loaded_model.predict(closed_prices_scaled)
  flattened_predictions = predictions.flatten()

  # Create a DataFrame with the 'elliot_wave' column
  df = pd.DataFrame({'elliot_wave': flattened_predictions})

  # Write the DataFrame to a CSV file
  df.to_csv(output_path, index=False)

  # count_greater_than_0_5 = np.sum(predictions > 0.5)
  # count_smaller_than_0_5 = np.sum(predictions < 0.5)
  # print(f'Count of values greater than 0.5: {count_greater_than_0_5}')
  # print(f'Count of values smaller than 0.5: {count_smaller_than_0_5}')

  max_p = max(predictions)
  index = np.argmax(predictions)

  most_likely = closed_prices_scaled[index]
  plot_most_likely(most_likely)


In [ ]:
# rename file paths to load the model, scaler and output path
elliot_predict(df, scaler, loaded_model, output_path)